# Language modelling

На данном семинаре мы:

1) поиграемся с игрушечной задачей. У нас есть выборка названий динозавров, попробуем генерировать новые названия по такому же формату. Обучим две различные символьные модели для генерации динозавров:
* модель на символьных биграмах
* ***RNN***-модель.
 
2) Рассмотрим инструменты для NER. Spacy, Natasha 

## Bigram model


In [1]:
!wget https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt

--2021-09-26 10:37:41--  https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19909 (19K) [text/plain]
Saving to: ‘dinos.txt’

dinos.txt           100%[===================>]  19.44K  --.-KB/s    in 0.001s  

2021-09-26 10:37:41 (35.0 MB/s) - ‘dinos.txt’ saved [19909/19909]



In [6]:
!head -n 10 dinos.txt

Aachenosaurus
Aardonyx
Abdallahsaurus
Abelisaurus
Abrictosaurus
Abrosaurus
Abydosaurus
Acanthopholis
Achelousaurus
Acheroraptor


In [12]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
print(names[:10])

['<aachenosaurus>', '<aardonyx>', '<abdallahsaurus>', '<abelisaurus>', '<abrictosaurus>', '<abrosaurus>', '<abydosaurus>', '<acanthopholis>', '<achelousaurus>', '<acheroraptor>']


In [27]:
# !pip3 install nltk
!pip install --user -U nltk

In [9]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров

In [10]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

In [11]:
print(list(freq.keys()))

['<', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', '>', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [12]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Определим функцию чтобы оценить вероятность символа. 

In [13]:
l = sum([freq[char] for char in freq])

def unigram_prob(char):
    return freq[char]/l

In [14]:
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1160


Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [16]:
cfreq = nltk.ConditionalFreqDist(
    nltk.bigrams(chars)
)

In [17]:
cfreq['a']

FreqDist({'>': 138,
          'a': 11,
          'b': 24,
          'c': 100,
          'd': 36,
          'e': 42,
          'f': 6,
          'g': 40,
          'h': 17,
          'i': 23,
          'j': 5,
          'k': 20,
          'l': 138,
          'm': 68,
          'n': 347,
          'o': 22,
          'p': 89,
          'q': 3,
          'r': 124,
          's': 171,
          't': 204,
          'u': 791,
          'v': 30,
          'w': 6,
          'x': 12,
          'y': 12,
          'z': 8})

Оценим условные вероятности с помощью Maximum Likelihood Estimator (MLE).

In [18]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [19]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [20]:
cprob['a'].generate()

'u'

### Task!
Напишите функцию которая генерирует имена динозавтров фикисрованной длины. Используйте '<' как стартовый символ и '>' как конечный.

## Solution

In [13]:
## 1
from nltk.util import bigrams, pad_sequence

# можно так ещё автоматически добавлять знаки начала / конца
list(pad_sequence(names[0], 
                  pad_left=False, 
                  left_pad_symbol="<",
                  pad_right=False, 
                  right_pad_symbol='>',
                  n=2)
)

['<', 'a', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '>']

In [1]:
from nltk.lm.preprocessing import padded_everygram_pipeline

dinos = []
with open('dinos.txt', 'r') as f:
    for line in f.readlines():
        dinos.append(list(line.strip()))

print(dinos[:5])

data, vocab = padded_everygram_pipeline(2, dinos)

[['A', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's'], ['A', 'a', 'r', 'd', 'o', 'n', 'y', 'x'], ['A', 'b', 'd', 'a', 'l', 'l', 'a', 'h', 's', 'a', 'u', 'r', 'u', 's'], ['A', 'b', 'e', 'l', 'i', 's', 'a', 'u', 'r', 'u', 's'], ['A', 'b', 'r', 'i', 'c', 't', 'o', 's', 'a', 'u', 'r', 'u', 's']]


In [2]:
print(len(dinos))
spl = int(90*len(dinos)/100)
train = dinos[:spl]
test = dinos[spl:]
len(train), len(test)

1536


(1382, 154)

In [3]:
# можно не писать руками доп символы и пэддинг а использовать готовую функцию
train_data, vocab = padded_everygram_pipeline(2, train)

In [4]:
from nltk.lm import MLE

lm = MLE(2) # 2 = наибольший размер используемых n-грамм
print(len(lm.vocab))
lm.fit(train_data, vocab)
len(lm.vocab)

0


49

In [8]:
lm.generate(4, random_seed=42)

['o', 'c', 'e', 'l']

In [9]:
# осторожно! если в обучении не было таких символов, то мы получим бесконечность
for t in train:
    t_ = ''.join(t)
    if t_.startswith('I'):
        print(t_)

print(lm.perplexity([('I', 'q')]))

Ichthyovenator
Ignavusaurus
Iguanacolossus
Iguanodon
Iguanoides
Iguanosaurus
Iliosuchus
Ilokelesia
Incisivosaurus
Indosaurus
Indosuchus
Ingenia
Inosaurus
Irritator
Isaberrysaura
Isanosaurus
Ischioceratops
Ischisaurus
Ischyrosaurus
Isisaurus
Issasaurus
Itemirus
Iuticosaurus
inf


In [14]:
padded_test = [['<']+t+['>'] for t in test]
print(*padded_test[:3], sep='\n')

bigrams_test = [list(bigrams(t)) for t in padded_test]
bigrams_test_flattened = []

for w in bigrams_test:
    for b in w:
        bigrams_test_flattened.append(b)
        
print(bigrams_test_flattened[:10])

lm.perplexity(bigrams_test_flattened[10:20])

['<', 'T', 'o', 'm', 'o', 'd', 'o', 'n', '>']
['<', 'T', 'o', 'n', 'g', 'a', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '>']
['<', 'T', 'o', 'n', 'g', 't', 'i', 'a', 'n', 'l', 'o', 'n', 'g', '>']
[('<', 'T'), ('T', 'o'), ('o', 'm'), ('m', 'o'), ('o', 'd'), ('d', 'o'), ('o', 'n'), ('n', '>'), ('<', 'T'), ('T', 'o')]


4.336383091572185

## Реккурентные нейронные сети (RNN)

Исходная последовательность:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

Для каждого входного значения $x_{1:i}$ получаем на выходе $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

Для всей последовательности $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ - рекурсивная функция активации, зависящая от двух параметров: $x_i$ и $s_{i-1}$ (вектор предыдущего состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- конкатенация $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

Построим языковую модель на основе RNN с помощью pytorch

In [15]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные

In [17]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]

        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [18]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [19]:
trn_ds.lines[1]

'aardonyx'

In [20]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [21]:
trn_ds.vocab_size

29

In [22]:
x, y = trn_ds[1]

In [23]:
x.shape

torch.Size([9, 29])

In [24]:
y.shape

torch.Size([9])

Опишем модель, функцию потерь и алгоритм оптимизации

In [25]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(combined)
        return h, y

In [26]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [27]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
 
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [32]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

Обучим получившуюся модель

In [36]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()
    return loss

In [39]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [40]:
train(model, loss_fn, optimizer, epochs=10)

Epoch:1
<puatosaurus>
<snrqsaurus>
<atgudauros>
<snccoaaurus>
<turaropturus>
<apsaisasaurus>
<apctrichurus>
<bustrasaurus>
<subsoapourus>
<bteboneurus>
<kvkviourus>
<rnwtpsosasiurus>
<fubcrecrauaoluitus>
<bustrastos>
<ltrrdapeor>

Epoch:2
<snsfniurus>
<gnbudaxtor>
<suraroptyth>
<tbntaisasionrus>
<llsopasah>
<ouctssnishurus>
<suanpalrhtn>
<aatiasaurus>
<antotaurus>
<qsgysgossgrus>
<scsatadooaaurus>
<scpussurus>
<dsomtrr>
<suailakimrus>
<burapeolarrus>

Epoch:3
<pucusdurus>
<yurosocaurus>
<smsaloparr>
<gnaudaurus>
<tuictorystor>
<appanoctosaat>
<apctpadter>
<oucososaurus>
<lsrtcepais>
<smsaiiurus>
<gnbudlor>
<butusaurus>
<lsrucoraurus>
<tcguraia>
<gactarysturus>

Epoch:4
<dhuiysisaurus>
<eubupudauros>
<smectanturus>
<spastitsplusaurrs>
<auktdastanaurus>
<antotaurus>
<rtnwsithuathn>
<scritaurus>
<cukodpllu>
<tautosaurus>
<lhuraphliusuurus>
<lbrisaurus>
<wrslsourus>
<kxsosaurus>
<eucthsaurus>

Epoch:5
<cugeeoasaurus>
<srcosaurus>
<tcqoceod>
<eubhricaurus>
<sdrurrliurus>
<rpttopoueaurus>
<s

## Named Entity Recognition


#### Постановка задачи «sequence labeling»:



* Дан корпус текстов $D$
* Каждый текст представляет собой последовательность токенов
* Каждому токену присвоена метка из некоторого множества $V$

В зависимости от множества меток $V$ получаем разные типы подзадач. Например:
* если $V$ - множество частей речи, то это задача ***POS***-теггинга
* если $V$ - множество типов именованных сущностей, то это задача ***NER***

Именованная сущность - любой фрагмент текста, обозначающий некоторый интересный объект.

In [41]:
!pip install natasha

     |████████████████████████████████| 34.4 MB 30 kB/s 
     |████████████████████████████████| 41 kB 113 kB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 19.1 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=c6245ef67dd725f9dabb3608c5837c021f71b79c10f8dc14d8d57333ed3f724d
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [42]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [43]:
!pip install html5lib

#### Spacy 

In [49]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [50]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


Выкачаем статью и найдём в ней именованные сущности, выведем их число:

In [53]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
len(article.ents)

153

Выведем число встреченных сущностей каждого типа:

In [52]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'CARDINAL': 3,
         'DATE': 23,
         'GPE': 9,
         'LOC': 1,
         'NORP': 2,
         'ORDINAL': 1,
         'ORG': 37,
         'PERSON': 77})

Выведем текст с подсвеченными сущностями разных типов:

In [54]:
sentences = [x for x in article.sents]
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

### Natasha

Natasha - это набор Python-библиотек для обработки текстов на естественном **русском** языке. Некоторые из них:

- Natasha - сегментация на токены и предложения, морфологический и синтаксический анализ, лемматизация, извлечение, нормализация именованных сущностей
- Yargy - парсер для извлечения структурированной информации из текстов, сспользует словари и правила
- Razdel - токензитор + сплиттер: делит текст на слова и предложения
- Slovnet - компактные модели для обработки естественного русского языка: морфологический теггер, синтаксический парсер, NER-теггер


Сайт: https://natasha.github.io/

In [55]:
import natasha
from natasha import Doc, NewsEmbedding, NewsNERTagger, MorphVocab

In [56]:
emb = NewsEmbedding()

ner_tagger = NewsNERTagger(emb)

In [58]:
text = '''Выпускница Гарварда Шерил Сэндберг ранее работала на руководящих должностях в министерстве финансов США и в Google, пришла в Facebook в 2008 году'''
markup = ner_tagger(text)
markup.print()

Выпускница Гарварда Шерил Сэндберг ранее работала на руководящих 
           ORG───── PER───────────                               
должностях в министерстве финансов США и в Google, пришла в Facebook в
                                   LOC     ORG───           ORG─────  
 2008 году


In [59]:
morph_vocab = MorphVocab() #обертка для Pymorphy2

morph_vocab('стекло')

[MorphForm(normal='стекло', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Neut', 'Number': 'Sing', 'Case': 'Nom'}),
 MorphForm(normal='стекло', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Neut', 'Number': 'Sing', 'Case': 'Acc'}),
 MorphForm(normal='стечь', pos='VERB', feats={'VerbForm': 'Fin', 'Aspect': 'Perf', 'Gender': 'Neut', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Ind'})]

In [63]:
from natasha import NamesExtractor

names_extractor = NamesExtractor(morph_vocab)

text = '''Генеральным директором группы компаний «Яндекс» сейчас является Аркадий Волож;
        основатель Илья Сегалович умер.'''
list(names_extractor(text))

[Match(
     start=12,
     stop=22,
     fact=Name(
         first=None,
         last='директором',
         middle=None
     )
 ), Match(
     start=40,
     stop=46,
     fact=Name(
         first=None,
         last='Яндекс',
         middle=None
     )
 ), Match(
     start=64,
     stop=77,
     fact=Name(
         first='Аркадий',
         last='Волож',
         middle=None
     )
 ), Match(
     start=98,
     stop=112,
     fact=Name(
         first='Илья',
         last='Сегалович',
         middle=None
     )
 )]

In [67]:
text = [
    'генеральный директор Эд Кэтмелл',
    'Джон Лассетер',
    'поэт А. С. Пушкин',
    'Марк Цукерберг'
]

for line in text:
    print(names_extractor.find(line))

Match(start=12, stop=20, fact=Name(first=None, last='директор', middle=None))
Match(start=0, stop=13, fact=Name(first='Джон', last='Лассетер', middle=None))
Match(start=5, stop=17, fact=Name(first='А', last='Пушкин', middle='С'))
Match(start=0, stop=14, fact=Name(first='Марк', last='Цукерберг', middle=None))


In [71]:
# DATES
from natasha import DatesExtractor

dates_extractor = DatesExtractor(morph_vocab)

text = '''
    24.01.2017, 2017 год, 2014 г, 1 апреля, май 2021 г., 
    9 мая 1945 года
    '''
print(*list(dates_extractor(text)), sep='\n')

Match(start=5, stop=15, fact=Date(year=2017, month=1, day=24))
Match(start=17, stop=25, fact=Date(year=2017, month=None, day=None))
Match(start=27, stop=33, fact=Date(year=2014, month=None, day=None))
Match(start=35, stop=43, fact=Date(year=None, month=4, day=1))
Match(start=45, stop=56, fact=Date(year=2021, month=5, day=None))
Match(start=63, stop=78, fact=Date(year=1945, month=5, day=9))


In [72]:
# MONEY
from natasha import MoneyExtractor

money_extractor = MoneyExtractor(morph_vocab)

text = '''$ 23, 100 рублей, 1,565,321 долларов, 34 тыс. евро, 
        тринадцать рублей тридцать две копейки, 13 руб. 32 коп.'''
print(*list(money_extractor(text)), sep='\n')

Match(start=2, stop=16, fact=Money(amount=23100, currency='RUB'))
Match(start=18, stop=36, fact=Money(amount=1565321, currency='USD'))
Match(start=38, stop=50, fact=Money(amount=34000, currency='EUR'))
Match(start=101, stop=116, fact=Money(amount=13.32, currency='RUB'))


In [77]:
# Adresses
from natasha import AddrExtractor

addr_extractor = AddrExtractor(morph_vocab)

lines = [
    'Россия, Москва, Кутузовский проспект, дом 45, корпус 1',
    '107845, РФ, Приморский край, г. Находка, ул. Добролюбова, 20',
    'поселок Солнечный, ул. Никитская, дом 10'
]
for line in lines:
    print(addr_extractor.find(line), end='\n\n')

Match(start=0, stop=54, fact=Addr(parts=[AddrPart(value='Россия', type='страна'), AddrPart(value='Москва', type=None), AddrPart(value='Кутузовский', type='проспект'), AddrPart(value='45', type='дом'), AddrPart(value='1', type='корпус')]))

Match(start=0, stop=56, fact=Addr(parts=[AddrPart(value='107845', type='индекс'), AddrPart(value='РФ', type='страна'), AddrPart(value='Приморский', type='край'), AddrPart(value='Находка', type='город'), AddrPart(value='Добролюбова', type='улица')]))

Match(start=0, stop=40, fact=Addr(parts=[AddrPart(value='Солнечный', type='посёлок'), AddrPart(value='Никитская', type='улица'), AddrPart(value='10', type='дом')]))

